In this notebook we will explore the traces.
We look at what kind of values appear for the different fields.
Also we hypothesize that there are some redundant field. So we will try to verify this.

In [1]:
import os
import sys
sys.path.append("../../src")
from features import trace_pre_processing as tpp
import json
from collections import Counter

In [2]:
# Lets find the paths to all the traces
trace_paths = []
for path_to_traces in ['../../data/raw/sequential_data/traces','../../data/raw/concurrent_data/traces']:
    trace_paths.extend([os.path.join(dp, f) for dp, dn, filenames in os.walk(path_to_traces) for f in filenames if os.path.splitext(f)[1] == '.json'])

In [3]:
# This method collects attributes values from the events using the collect_function. It does that for all the available events and returns them either as a list or a set depending on the parameter unique.
def find_attribute_values(collect_function, unique=False):
    occuring_values = list()
    for trace_path in trace_paths:
        with open(trace_path) as trace_file:
            trace_json = json.load(trace_file)
            values_for_trace = tpp.get_flat_list(trace_json,collect_function)
            occuring_values.extend(values_for_trace)
    if unique:
        return list(filter(lambda x: x != None, set(occuring_values)))
    else:
        return list(filter(lambda x: x != None, occuring_values))


# This method prints the elements
def print_information(values, msg='Information about the values in the list:', print_counts = True):
    print(msg)
    print('A total of ' + str(len(values)) + ' values were collected')
    if print_counts:
        counts = Counter(values).items()
        print('A total of ' + str(len(counts)) + ' unique values were collected')
        for e, c in sorted(counts, key=lambda x: x[1], reverse=True):
            print(str(c) + ' times: ' + str(e))
    else:
        for v in values:
            print(str(v))

First of all we are interested in some of the basic attributes of events

In [4]:
def collect_interesting_attributes(event):
    return [
        [
            event['info'].get('name'), 
            event['info'].get('service'), 
            event['info'].get('project'), 
            event['info'].get('exception'),
            event['info'].get('host')
        ]
    ]

basic_event_attributes = find_attribute_values(collect_interesting_attributes)
print_information(list(map(lambda x: x[0], basic_event_attributes)), 'Information about the name field')
print_information(list(map(lambda x: x[1], basic_event_attributes)), '\nInformation about the service field')
print_information(list(map(lambda x: x[2], basic_event_attributes)), '\nInformation about the project field')
print_information(list(map(lambda x: x[3], basic_event_attributes)), '\nInformation about the exception field')
print_information(list(map(lambda x: x[4], basic_event_attributes)), '\nInformation about the host field')

Information about the name field
A total of 2519693 values were collected
A total of 7 unique values were collected
979001 times: neutron.db
659774 times: db
592508 times: neutron_api
167904 times: wsgi
77706 times: rpc
40648 times: compute_api
2152 times: nova_image

Information about the service field
A total of 2519693 values were collected
A total of 8 unique values were collected
1033490 times: neutron-server
633158 times: osapi_compute
286439 times: public
245013 times: api
243224 times: nova-conductor
47153 times: nova-compute
25073 times: nova-scheduler
6143 times: neutron-dhcp-agent

Information about the project field
A total of 2519693 values were collected
A total of 4 unique values were collected
1039633 times: neutron
948608 times: nova
286439 times: keystone
245013 times: glance

Information about the exception field
A total of 2519693 values were collected
A total of 5 unique values were collected
2514661 times: None
4060 times: keystoneauth1.exceptions.connection.Conne

Lets look at some of the combinations of these attributes as well:

In [5]:
print_information(list(map(lambda x: (x[0], x[1]), basic_event_attributes)), 'Information about the combination of the name and service field')
print_information(list(map(lambda x: (x[0], x[4]), basic_event_attributes)), '\nInformation about the combination of the name and host field')
print_information(list(map(lambda x: (x[1], x[4]), basic_event_attributes)), '\nInformation about the combination of the service and host field')

Information about the combination of the name and service field
A total of 2519693 values were collected
A total of 19 unique values were collected
979001 times: ('neutron.db', 'neutron-server')
551189 times: ('neutron_api', 'osapi_compute')
238350 times: ('db', 'public')
217419 times: ('db', 'api')
203695 times: ('db', 'nova-conductor')
54489 times: ('wsgi', 'neutron-server')
48089 times: ('wsgi', 'public')
41319 times: ('neutron_api', 'nova-compute')
40648 times: ('compute_api', 'osapi_compute')
39529 times: ('rpc', 'nova-conductor')
37732 times: ('wsgi', 'osapi_compute')
27594 times: ('wsgi', 'api')
24763 times: ('rpc', 'nova-scheduler')
6143 times: ('rpc', 'neutron-dhcp-agent')
5830 times: ('rpc', 'nova-compute')
2148 times: ('nova_image', 'osapi_compute')
1441 times: ('rpc', 'osapi_compute')
310 times: ('db', 'nova-scheduler')
4 times: ('nova_image', 'nova-compute')

Information about the combination of the name and host field
A total of 2519693 values were collected
A total of 16

Lets have a closer look at the payload attribute
My observation is that events contain two fields holding payload data: meta_raw_payload.name-start and meta_raw_payload.name-stop
We need to verify that every event contains both fields.

In [6]:
def is_event_empty_or_top_level(event):
    return len(event) == 0 or event['info']['name'] == 'total'

def get_payload_start_name(event):
    return 'meta.raw_payload.' + event['info']['name'] + '-start'

def get_payload_stop_name(event):
    return 'meta.raw_payload.' + event['info']['name'] + '-stop'

def does_event_contain_both_payload_fields(event):
    return event['info'].get(get_payload_start_name(event)) != None and event['info'].get(get_payload_stop_name(event)) != None

def collect_trace_ids_for_events_that_dont_have_both_payload_fields(event):
    if is_event_empty_or_top_level(event) or does_event_contain_both_payload_fields(event): return [None] 
    return [(event['info']['name'], event['trace_id'], event['info'].get('service'))]

print_information(find_attribute_values(collect_trace_ids_for_events_that_dont_have_both_payload_fields, unique=True), msg="Events that don't contain both payload fields", print_counts=False)

Events that don't contain both payload fields
A total of 70 values were collected
('wsgi', '4434e85d-a739-4e53-abb1-eb8f61becdbc', 'neutron-server')
('wsgi', 'cf9d2296-09aa-43bc-a983-472c429b9be7', 'api')
('wsgi', '36008481-29ca-47d5-9268-27b9549cebf4', 'api')
('wsgi', '3587372d-d48c-4826-9780-c1ef3d547515', 'api')
('wsgi', '88af3ac3-d3a9-43b8-8663-aca75b20e0ba', 'api')
('neutron.db', '2a8305fe-4141-4c83-8976-82bd3e250637', 'neutron-server')
('wsgi', '92ab4b65-0db6-451b-8203-1a4f8d7e6838', 'neutron-server')
('wsgi', 'cbbc0733-2a24-42c6-a6c0-9429c3212f16', 'neutron-server')
('neutron.db', '2aa71ef7-5f96-4997-ab4f-69e14b2f78be', 'neutron-server')
('wsgi', '6d1677ec-39f5-4fa3-8852-c7c15f078ab4', 'api')
('wsgi', '3ab8b155-010d-4688-be5c-ae12bbc40d29', 'api')
('wsgi', '2de4db85-7446-4c9d-b958-0312b714491d', 'api')
('wsgi', 'eb18b4ec-e6c6-4e0b-92ee-cb36184b799f', 'api')
('db', 'f38a99fc-c5f5-4863-9f6e-4febb095ea76', 'api')
('wsgi', '3124750e-90e6-44db-804d-a0fe411ed218', 'neutron-server')
('

For an event with both payload fields (payload-start and payload-stop) there exist some redundant fields.
We want to determine whether the fields have the same values.
We are looking at the following fields: host, service and project

In [7]:
def are_redundant_fields_equal(event):
    info_host = event['info'].get('host')
    info_service = event['info'].get('service')
    info_project = event['info'].get('project')
    assert info_host != None
    assert info_service != None
    assert info_project != None
    
    payload_start_name = get_payload_start_name(event)
    payload_stop_name = get_payload_stop_name(event)
    
    return info_host == event['info'][payload_start_name]['info']['host'] == event['info'][payload_stop_name]['info']['host'] and info_service == event['info'][payload_start_name]['service'] == event['info'][payload_stop_name]['service'] and info_project == event['info'][payload_start_name]['project'] == event['info'][payload_stop_name]['project']

def collect_events_where_values_in_redudant_fields_differ(event):
    if is_event_empty_or_top_level(event) or does_event_contain_both_payload_fields(event) == False: return [None] 
    if are_redundant_fields_equal(event):
        return [None]
    else:
        return [(name, event['trace_id'], event['info'].get('service'), info_host, payload_start_host, payload_stop_host)]
    
print_information(find_attribute_values(collect_events_where_values_in_redudant_fields_differ, True), 'Events where values in redundant fields differ', print_counts=False)

Events where values in redundant fields differ
A total of 0 values were collected


We want to better understand the payload field. It contains different fields based on what kind of service is executed. So lets try to find out what kind of fields exist for each service.

In [8]:
# This function collects the keys of a dictionary recursively as a string representation
def collect_keys_recursively(payload_dict, earlier_keys = ''):
    if len(payload_dict) == 0: return earlier_keys
        
    for k,v in payload_dict.items():
        if len(earlier_keys) and earlier_keys[-1] != '-' and earlier_keys[-1] != '[':
            earlier_keys += '-'
        earlier_keys += str(k)
        if type(v) is dict:
            earlier_keys += '['
            earlier_keys = collect_keys_recursively(v, earlier_keys)
            earlier_keys += ']'
        
    return earlier_keys


def collect_keys_of_payload_fields(event):
    if is_event_empty_or_top_level(event) or does_event_contain_both_payload_fields(event) == False : 
        return [None]
    
    payload_start = event['info'][get_payload_start_name(event)]
    payload_stop = event['info'][get_payload_stop_name(event)]
    
    return [
        (
            event['info'].get('name'), 
            event['info'].get('service'),
            event['info'].get('project'),
            event['info'].get('host'),
            get_payload_start_name(event), 
            get_payload_stop_name(event), 
            collect_keys_recursively(payload_start), 
            collect_keys_recursively(payload_stop)
        )
    ]

event_attributes = find_attribute_values(collect_keys_of_payload_fields)
print_information(event_attributes)

Information about the values in the list:
A total of 2519623 values were collected
A total of 324 unique values were collected
547799 times: ('neutron_api', 'osapi_compute', 'nova', 'wally113', 'meta.raw_payload.neutron_api-start', 'meta.raw_payload.neutron_api-stop', 'info[function[args-name-kwargs]-host]-name-service-timestamp-trace_id-project-parent_id-base_id', 'info[host]-name-service-timestamp-trace_id-project-parent_id-base_id')
181562 times: ('neutron.db', 'neutron-server', 'neutron', 'wally113', 'meta.raw_payload.neutron.db-start', 'meta.raw_payload.neutron.db-stop', 'info[host-db[params[target_tenant_1-target_tenant_2-target_tenant_3-target_tenant_4-id_1-action_2-action_1-project_id_1]-statement]]-name-service-timestamp-trace_id-project-parent_id-base_id', 'info[host]-name-service-timestamp-trace_id-project-parent_id-base_id')
167032 times: ('neutron.db', 'neutron-server', 'neutron', 'wally113', 'meta.raw_payload.neutron.db-start', 'meta.raw_payload.neutron.db-stop', 'info[ho

In [9]:
print_information(list(map(lambda x : x[6], event_attributes)), 'Different payload-start features')

Different payload-start features
A total of 2519623 values were collected
A total of 290 unique values were collected
713014 times: info[function[args-name-kwargs]-host]-name-service-timestamp-trace_id-project-parent_id-base_id
351168 times: info[host-db[params[]-statement]]-name-service-timestamp-trace_id-project-parent_id-base_id
223553 times: info[host-db[params[param_1]-statement]]-name-service-timestamp-trace_id-project-parent_id-base_id
222188 times: info[host-db[params[id_1]-statement]]-name-service-timestamp-trace_id-project-parent_id-base_id
181562 times: info[host-db[params[target_tenant_1-target_tenant_2-target_tenant_3-target_tenant_4-id_1-action_2-action_1-project_id_1]-statement]]-name-service-timestamp-trace_id-project-parent_id-base_id
167843 times: info[host-request[path-scheme-method-query]]-name-service-timestamp-trace_id-project-parent_id-base_id
81915 times: info[host-db[params[is_dynamic_1-network_id_1]-statement]]-name-service-timestamp-trace_id-project-parent_id

In the earlier output we could see that the features mainly vary because the params field contain different keys.
These are the params that get passed to a database query. Im going to adapt our collect function to not further descend into the dict if it is the 'params' field:

In [10]:
def collect_keys_recursively_exclude_params(payload_dict, earlier_keys = ''):
    if len(payload_dict) == 0: return earlier_keys
        
    for k,v in payload_dict.items():
        if len(earlier_keys) and earlier_keys[-1] != '-' and earlier_keys[-1] != '[':
            earlier_keys += '-'
        earlier_keys += str(k)
        if type(v) is dict and str(k) != 'params':
            earlier_keys += '['
            earlier_keys = collect_keys_recursively_exclude_params(v, earlier_keys)
            earlier_keys += ']'
        
    return earlier_keys

def collect_keys_of_payload_fields_exclude_param(event):
    if is_event_empty_or_top_level(event) or does_event_contain_both_payload_fields(event) == False : 
        return [None]
    
    payload_start = event['info'][get_payload_start_name(event)]
    payload_stop = event['info'][get_payload_stop_name(event)]
    
    return [
        (
            event['info'].get('name'), 
            event['info'].get('service'),
            event['info'].get('project'),
            event['info'].get('host'),
            get_payload_start_name(event), 
            get_payload_stop_name(event), 
            collect_keys_recursively_exclude_params(payload_start), 
            collect_keys_recursively_exclude_params(payload_stop)
        )
    ]

event_attributes_exclude_params_field = find_attribute_values(collect_keys_of_payload_fields_exclude_param)
print_information(event_attributes_exclude_params_field)

Information about the values in the list:
A total of 2519623 values were collected
A total of 30 unique values were collected
978997 times: ('neutron.db', 'neutron-server', 'neutron', 'wally113', 'meta.raw_payload.neutron.db-start', 'meta.raw_payload.neutron.db-stop', 'info[host-db[params-statement]]-name-service-timestamp-trace_id-project-parent_id-base_id', 'info[host]-name-service-timestamp-trace_id-project-parent_id-base_id')
547799 times: ('neutron_api', 'osapi_compute', 'nova', 'wally113', 'meta.raw_payload.neutron_api-start', 'meta.raw_payload.neutron_api-stop', 'info[function[args-name-kwargs]-host]-name-service-timestamp-trace_id-project-parent_id-base_id', 'info[host]-name-service-timestamp-trace_id-project-parent_id-base_id')
238350 times: ('db', 'public', 'keystone', 'wally113', 'meta.raw_payload.db-start', 'meta.raw_payload.db-stop', 'info[host-db[params-statement]]-name-service-timestamp-trace_id-project-parent_id-base_id', 'info[host]-name-service-timestamp-trace_id-proj

In [11]:
print_information(list(map(lambda x: x[6], event_attributes_exclude_params_field)), 'Different payload-start features if we do not descend inside the params dict')

Different payload-start features if we do not descend inside the params dict
A total of 2519623 values were collected
A total of 3 unique values were collected
1638766 times: info[host-db[params-statement]]-name-service-timestamp-trace_id-project-parent_id-base_id
713014 times: info[function[args-name-kwargs]-host]-name-service-timestamp-trace_id-project-parent_id-base_id
167843 times: info[host-request[path-scheme-method-query]]-name-service-timestamp-trace_id-project-parent_id-base_id


In [12]:
print_information(list(map(lambda x : x[7], event_attributes)), 'Different payload-stop features')

Different payload-stop features
A total of 2519623 values were collected
A total of 2 unique values were collected
2514661 times: info[host]-name-service-timestamp-trace_id-project-parent_id-base_id
4962 times: info[etype-message-host]-name-service-timestamp-trace_id-project-parent_id-base_id


No params field here, so this will be the same for both collect-keys functions. The second feature appears when an exception occured

In [13]:
print_information(list(map(lambda x : (x[6], x[7]), event_attributes_exclude_params_field)), 'Different payload-start/ payload-stop feature combination')

Different payload-start/ payload-stop feature combination
A total of 2519623 values were collected
A total of 4 unique values were collected
1638766 times: ('info[host-db[params-statement]]-name-service-timestamp-trace_id-project-parent_id-base_id', 'info[host]-name-service-timestamp-trace_id-project-parent_id-base_id')
708052 times: ('info[function[args-name-kwargs]-host]-name-service-timestamp-trace_id-project-parent_id-base_id', 'info[host]-name-service-timestamp-trace_id-project-parent_id-base_id')
167843 times: ('info[host-request[path-scheme-method-query]]-name-service-timestamp-trace_id-project-parent_id-base_id', 'info[host]-name-service-timestamp-trace_id-project-parent_id-base_id')
4962 times: ('info[function[args-name-kwargs]-host]-name-service-timestamp-trace_id-project-parent_id-base_id', 'info[etype-message-host]-name-service-timestamp-trace_id-project-parent_id-base_id')


In [14]:
print_information(list(map(lambda x: (x[0], x[6]), event_attributes_exclude_params_field)), 'Combination of name and payload-start features')

Combination of name and payload-start features
A total of 2519623 values were collected
A total of 7 unique values were collected
978997 times: ('neutron.db', 'info[host-db[params-statement]]-name-service-timestamp-trace_id-project-parent_id-base_id')
659769 times: ('db', 'info[host-db[params-statement]]-name-service-timestamp-trace_id-project-parent_id-base_id')
592508 times: ('neutron_api', 'info[function[args-name-kwargs]-host]-name-service-timestamp-trace_id-project-parent_id-base_id')
167843 times: ('wsgi', 'info[host-request[path-scheme-method-query]]-name-service-timestamp-trace_id-project-parent_id-base_id')
77706 times: ('rpc', 'info[function[args-name-kwargs]-host]-name-service-timestamp-trace_id-project-parent_id-base_id')
40648 times: ('compute_api', 'info[function[args-name-kwargs]-host]-name-service-timestamp-trace_id-project-parent_id-base_id')
2152 times: ('nova_image', 'info[function[args-name-kwargs]-host]-name-service-timestamp-trace_id-project-parent_id-base_id')


We have 7 unique names and 7 unique combinations of name and payload-start features. **Therefore is the name field sufficient to infer what kind of features the payload-start field contains.**
The exception can also be infered from the payload.